In [1]:
import os
import re
import json
import glob
import random
from collections import defaultdict

from functools import partial
from pathlib import Path
from tqdm import tqdm

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import nltk
import string

# spaCy 
import spacy
from spacy.util import minibatch, compounding
#from spacy.training.example import Example # version 3 only

KAGGLE = False

## Preprocessing

In [2]:
if KAGGLE:
    from model.model import longest_consecutive_caps as LCC
    from model.model import KMP

    train_df = pd.read_csv("/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv")
    sample_sub = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
    train_fp = "/kaggle/input/coleridgeinitiative-show-us-the-data/train/"
    test_fp = "/kaggle/input/coleridgeinitiative-show-us-the-data/test/"

else:
    from model import longest_consecutive_caps as LCC
    from model import KMP
    
    train_df = pd.read_csv("dataset/train.csv")
    sample_sub = pd.read_csv('dataset/sample_submission.csv')
    train_fp = "dataset/train/"
    test_fp = "dataset/test/"

In [3]:
#!python -m spacy download en_core_web_trf

In [4]:
#train_df.head(5)

In [5]:
#train_df.info()

In [6]:
def read_append_return(filename, train_files_path=train_fp, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [7]:
%%time
tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
train_df['text'] = train_df['Id'].progress_apply(read_append_return)

/home/ry/.local/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 19661/19661 [00:05<00:00, 3778.86it/s]CPU times: user 4.4 s, sys: 807 ms, total: 5.21 s
Wall time: 5.21 s



In [8]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    #text = re.sub(' +', ' ', str(text).lower()).strip()
    text = re.sub('[^A-Za-z0-9 ]+', '|', str(text).lower())
    #text = ''.join([k for k in text if k not in string.punctuation])
    #text = re.sub('[^A-Za-z0-9.]+', ' ', str(text).lower()).strip()
#     text = re.sub("/'+/g", ' ', text)
    return text

In [9]:
%%time
tqdm.pandas()
train_df['text'] = train_df['text'].progress_apply(text_cleaning)

100%|██████████| 19661/19661 [00:34<00:00, 574.80it/s] CPU times: user 33.6 s, sys: 824 ms, total: 34.4 s
Wall time: 34.2 s



In [10]:
a = train_df["Id"].nunique()
b = train_df["Id"].size
print(a, b)
print(a/b)

14316 19661
0.7281420070189716


In [11]:
#train_df.head()

## Training
### Choose spaCy model and prepare training data

In [12]:
# data configurations

# select model backbone
model = None 
#model = "specified" # specified for transformer + ner only
#model = "generic" # a model based on existing en_core_web_sm

token_anno = "entities" # note this may require changes for v3.0+ transformer models
negative_sample = True # turn on or off negative sampling

In [13]:
TRAIN_DATA = []
x1, x2, x3, x4 = 0, 0, 0, 0

print("Preparing training data...")

for index, row in tqdm(train_df.iterrows()):
    # get text of each sample test
    train_text = row['text']
    row_id = row['Id']
    label = row['cleaned_label']
    m = len(label)
    text = train_text.lower().split('|')
    #text = re.split('[?.,;\n\t&!()]+', train_text) # can't have sample_text.lower() since I need to find consecutive caps

    # begin matching
    for sentence in text:
        x1 += 1
        # clean text 
        #sentence = clean_text(sentence)
        indexed = KMP(label, sentence)
        if indexed != []:
            n = len(sentence)
            elist = []
            for i in indexed:
                end = m + i
                a = ((end < n and sentence[end] == ' ') or end >= n) # can have NEGATIVE SAMPLING like "ADNI-2" (or are those negative ones?)
                b = ((i > 0 and sentence[i-1] == ' ') or i == 0)
                if a and b:
                    entity = (i, end, "DATASET") 
                    elist.append(entity)
                    x3 += 1
            if elist == []: break # TEST (REMOVE IF NEEDED)
            x = (sentence, {token_anno:elist})
            TRAIN_DATA.append(x)
        
        elif negative_sample: 
            x2 += 1
            if random.random() > 0.995:
                x4 += 1
                TRAIN_DATA.append((sentence, {token_anno:[]})) # TEST

19661it [06:35, 49.69it/s]


In [14]:
print("[INFO]")
print("Among %d sentences,"%x1)
print("%d sentences have no title in them,"%x2)
print("meaning %.3f%% do not have title."%(x2/x1*100))
print("So we have %d sentences with titles."%(x1-x2))
print("In %d sentences, we obtained %d tokens (positive samples) that perfectly match titles."%(x1-x2, x3))
print("For sentences without any title, we chose %d negative samples."%x4)
print("The ratio of positive vs negative samples is %.2f : 100."%(x3 / x4 * 100))


[INFO]
Among 35961857 sentences,
35908537 sentences have no title in them,
meaning 99.852% do not have title.
So we have 53320 sentences with titles.
In 53320 sentences, we obtained 53506 tokens (positive samples) that perfectly match titles.
For sentences without any title, we chose 358706 negative samples.
The ratio of positive vs negative samples is 14.92 : 100.


### Configure training pipeline and train the spaCy model

Download official model with `python3 -m spacy download en_core_web_sm`

In [ ]:
# training configurations
output_dir = "output/"
n_iter = 15 # number of training iteration

In [15]:
print("Using GPU:", spacy.prefer_gpu())

# load the model
if model == "specified":
    nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])
elif model == "generic":
    nlp = spacy.load("en_core_web_sm")
    print("Loaded generic model en_core_web_sm")
elif model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

# add ner component to pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner') # version < 3.0
    nlp.add_pipe(ner, last=True) # verions < 3.0
    #nlp.add_pipe("ner", last=True) # version >= 3.0 only
    #ner = nlp.get_pipe("ner") # version >= 3.0 only
else:
    ner = nlp.get_pipe('ner')

# add all labels to the ner
#for _, annotations in TRAIN_DATA:
#    for ent in annotations.get('entities'):
#        ner.add_label(ent[2])
ner.add_label("DATASET") # only one category so no for loop

# configure optimizer (may not work with spaCy v3 transformers)
optimizer = nlp.begin_training()

# configure pipeline components to disable
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

Using GPU: True
Created blank 'en' model


In [16]:
# start training
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001)) # HYPERPARAMETERS
        # version < 3.0 use this
        for batch in tqdm(batches):
            texts, annotations = zip(*batch)
            nlp.update(
                texts, # or [texts] if not using batch
                annotations, # or [annotations] if not using batch
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        
        # For spaCy v3.0+ use the similar for loops in archived.ipynb
            
        print("\titeration %d, ner loss: %.2f"%(itn+1, losses['ner']))

14092it [08:32, 27.50it/s]
0it [00:00, ?it/s]	iteration 1, ner loss: 16403.95
14092it [08:27, 27.78it/s]
0it [00:00, ?it/s]	iteration 2, ner loss: 11636.79
14092it [08:20, 28.14it/s]
	iteration 3, ner loss: 10677.45
14092it [07:57, 29.51it/s]
0it [00:00, ?it/s]	iteration 4, ner loss: 10381.32
14092it [07:55, 29.65it/s]
0it [00:00, ?it/s]	iteration 5, ner loss: 10086.71
14092it [08:41, 27.02it/s]
	iteration 6, ner loss: 10257.75
14092it [09:10, 25.60it/s]
	iteration 7, ner loss: 10441.79
14092it [09:13, 25.48it/s]
	iteration 8, ner loss: 10276.88
14092it [09:02, 26.00it/s]
	iteration 9, ner loss: 10391.56
14092it [09:03, 25.94it/s]
	iteration 10, ner loss: 10218.74
14092it [09:03, 25.94it/s]
	iteration 11, ner loss: 10218.12
14092it [08:54, 26.38it/s]
	iteration 12, ner loss: 10324.05
14092it [08:45, 26.81it/s]
	iteration 13, ner loss: 10320.37
14092it [08:12, 28.62it/s]
0it [00:00, ?it/s]	iteration 14, ner loss: 10401.77
14092it [08:04, 29.06it/s]
0it [00:00, ?it/s]	iteration 15, ner l

## Testing and Exporting Model

In [17]:
# sample test
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    break

# save model
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Entities []
Saved model to output


## Inference with Trained Model
Generates the submission file

In [18]:
def clean_text(txt):
    ''' DO NOT DELETE: Official function for submission text cleaning '''
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()
    #return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [19]:
%%time
tqdm.pandas()
sample_sub['text'] = sample_sub['Id'].progress_apply(partial(read_append_return, train_files_path=test_fp))

/home/ry/.local/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 4/4 [00:00<00:00, 511.69it/s]CPU times: user 5.96 ms, sys: 3.97 ms, total: 9.93 ms
Wall time: 11.8 ms



In [20]:
# STRING MATCHING BLOCK
temp_1 = [x.lower() for x in train_df['dataset_label'].unique()]
temp_2 = [x.lower() for x in train_df['dataset_title'].unique()]
temp_3 = [x.lower() for x in train_df['cleaned_label'].unique()]

existing_labels = set(temp_1 + temp_2 + temp_3)
id_list = []
lables_list = []

# load model 'EN'
#nlp2 = spacy.load(output_dir) # loading an model can be slower?

for index, row in tqdm(sample_sub.iterrows()):
    # get text of each sample test
    sample_text = row['text']
    row_id = row['Id']
    temp_df = train_df[train_df['text'] == text_cleaning(sample_text)]

    cleaned_labels = temp_df['cleaned_label'].to_list()

    texts = sample_text.lower().split('.')
    #text = re.split('[?.,;\n\t&!]', sample_text) # can't have sample_text.lower() since I need to find consecutive caps

    # begin search
    # matching
    
    for known_label in existing_labels:   # for each label in the known set
        # EXACT MATCH
        if known_label in sample_text.lower():   # find the EXACT label in text 
            cleaned_labels.append(clean_text(known_label)) # if found, then append to the list for further formatting
    
    # THIS METHOD BELOW IS MUCH FASTER!!!!!
    # SOURCE: https://spacy.io/usage/processing-pipelines
    # SOURCE 2: https://prrao87.github.io/blog/spacy/nlp/performance/2020/05/02/spacy-multiprocess.html
    # THANK YOU SOOO MUCH!!!
    # Disabling pipeline components also helps
    for doc in nlp.pipe(texts, disable=other_pipes):
        cleaned_labels.extend([clean_text(entity.text) for entity in doc.ents if entity.label_ == "DATASET"])
    

    #for sentence in texts:   
        #doc = nlp(text_cleaning(sentence))
    #    doc = nlp2(text_cleaning(sentence))

    #    for entity in doc.ents:
    #        if entity.label_ == 'DATASET':
    #            cleaned_labels.append(clean_text(entity.text))   
            
        # CASE 1: FUZZYMATCH (see archived)

        # CASE 2: Consecutive Capitalizations (see archived)
        
        
    #cleaned_labels = [clean_text(x) for x in cleaned_labels]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(row_id)

4it [00:00,  4.00it/s]


In [21]:
submission = pd.DataFrame()
submission['Id'] = id_list
submission['PredictionString'] = lables_list

In [22]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
submission.head()

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,s|adni|alzheimer s disease neuroimaging initia...
1,2f392438-e215-4169-bebf-21ac4ff253e1,school teacher salaries in|3 to|s|isco 88 inte...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,8 00|s|slosh meows|slosh moms|slosh|slosh mode...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,rural urban continuum codes|2008 see fig|s


In [23]:
submission.to_csv('submission.csv', index=False)

In [24]:
for item in submission["PredictionString"]:
    print(item)
    print()

s|adni|alzheimer s disease neuroimaging initiative adni|47 10|alzheimer s disease neuroimaging initiative adni |8

school teacher salaries in|3 to|s|isco 88 international|trends in international mathematics and science study|nces common core of data|common core of data

8 00|s|slosh meows|slosh moms|slosh|slosh model|coastal observation station|noaa storm surge inundation|3 00|sea lake and overland surges from hurricanes|noaa 2016 and|slosh and|coastal change science along

rural urban continuum codes|2008 see fig|s

